# Data preparation

After data translation and data curation, the dataset needs to be prepared for model creation. In previous steps we noticed that the dataset is not ready to be processed due to various reasons.

The scope of this notebook is to eliminate any missing values, create any remaining dummy variables and drop features (columns) that won't be used.

## Load data

In [1]:
import pandas as pd
import os

In [2]:
input_file_path = 'data/data_curated.csv'
data_curated = pd.read_csv(input_file_path)

data_curated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 68 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   TimeStamp             580 non-null    object 
 1   YearsExperience       580 non-null    object 
 2   DevelopmentType       580 non-null    object 
 3   ProgrammingLanguages  580 non-null    object 
 4   CompanySize           580 non-null    object 
 5   WorkLocation          580 non-null    object 
 6   SuperivisionRole      580 non-null    float64
 7   WorkOutsideMainJob    574 non-null    float64
 8   CityLive              579 non-null    object 
 9   CityWork              282 non-null    object 
 10  Sex                   571 non-null    object 
 11  NetSalary             580 non-null    float64
 12  ai / ml               580 non-null    float64
 13  backend               580 non-null    float64
 14  bi                    580 non-null    float64
 15  data                  5

We should focus on columns that have an `object` Dtype. Those are `TimeStamp`, `YearsExperience`, `DevelopmentType`, `ProgrammingLanguages`, `CompanySize`, `WorkLocation`, `CityLive`, `CityWork` and `Sex`.

## Fill missing values

We see that the `WorkOutsideMainJob` column has 6 missing values.

In [3]:
data_curated.WorkOutsideMainJob.astype('category').value_counts(dropna=False)

0.0    394
1.0    180
NaN      6
Name: WorkOutsideMainJob, dtype: int64

We replace the missing values by assing a negative semantic value (zero). We interpret the absense of an answer as negation.

In [4]:
data_curated.WorkOutsideMainJob = data_curated.WorkOutsideMainJob.fillna(0.)

The `CityLive` value contains only 1 missing value.

In [5]:
data_curated.CityLive.astype('category').value_counts(dropna=False)

Athena          446
Thessaloniki     53
Heraklion        11
Patras            8
Amsterdam         7
Janina            6
Serres            5
London            4
Berlin            4
Bratislava        2
Kavala            2
Kozani            2
Salamis           2
Cyprus            1
Drama             1
Chania            1
Glasgow           1
Cologne           1
Chios             1
Kalmar            1
Kalymnos          1
Karlskrona        1
Eindhoven         1
NaN               1
Komotini          1
Oxford            1
Volos             1
Vienna            1
Tripoli           1
Stockholm         1
Rethimno          1
Piraeus           1
Nuremberg         1
Krakow            1
Munich            1
Malmö             1
Madrid            1
Limassol          1
Leeds             1
Xanthi            1
Larissa           1
Name: CityLive, dtype: int64

We examine the specific row.

In [6]:
data_curated[data_curated.CityLive.isna()]

,TimeStamp,YearsExperience,DevelopmentType,ProgrammingLanguages,CompanySize,WorkLocation,SuperivisionRole,WorkOutsideMainJob,CityLive,CityWork,...,ruby,rust,scala,sql,swift,tcl,typescript,vb,vue,wordpress
200,7/22/2020 23:58:21,11+,"Backend, Frontend",JavaScript,51-100,Remote,0.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
data_curated[data_curated.CityLive.isna()].NetSalary

200    104000.0
Name: NetSalary, dtype: float64

The case can be considered as an outlier & since it does not contain a lot of information it is dropped.

In [8]:
data_curated = data_curated.drop(200)

## Create dummy variables

Since the columns `YearsExperience`, `CompanySize`, `WorkLocation` amd `CityLive` can be considered as categorical variables, they should be converted to dummy variables.

In [9]:
data_curated = pd.get_dummies(data_curated,columns=['YearsExperience'])
data_curated = pd.get_dummies(data_curated,columns=['CompanySize'])
data_curated = pd.get_dummies(data_curated,columns=['WorkLocation'])
data_curated = pd.get_dummies(data_curated,columns=['CityLive'])

## Drop variables

In the last step, it is decided that the following columns should be dropped. `TimeStamp` does not offer any useful information. The `DevelopmentType` and `ProgrammingLanguages` columns were converted into dummy variables in a previous step. `CityWork` contains too many missing values. Finally, `Sex` is dropped due to its uneven distribution. This might introduce bias against women/females.

In [10]:
data_curated = data_curated.drop(labels=['TimeStamp','DevelopmentType','ProgrammingLanguages','CityWork','Sex'], axis=1)

The final list of columns is printed below.

In [11]:
data_curated.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 579 entries, 0 to 579
Data columns (total 113 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   SuperivisionRole                     579 non-null    float64
 1   WorkOutsideMainJob                   579 non-null    float64
 2   NetSalary                            579 non-null    float64
 3   ai / ml                              579 non-null    float64
 4   backend                              579 non-null    float64
 5   bi                                   579 non-null    float64
 6   data                                 579 non-null    float64
 7   desktop                              579 non-null    float64
 8   devops                               579 non-null    float64
 9   drupal                               579 non-null    float64
 10  embedded                             579 non-null    float64
 11  frontend                       

## Save data

In [12]:
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

output_file = 'data_ready.csv'
data_curated.to_csv(os.path.join(data_dir, output_file), index=False)